In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import datetime as dt
from ignite.engine import Engine
from ignite.metrics import MeanSquaredError
from ignite.engine import Events
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [25]:
class NeuralNetwork(nn.Module):
    def __init__(self, inputDim):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(inputDim, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

class training_set(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.X)

In [26]:
from ignite.contrib.handlers import TensorboardLogger, global_step_from_engine
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.metrics.regression import R2Score

In [33]:
def trainStep(engine, batch):
    model.train()
    X, y = batch

    optimizer.zero_grad()
    yPred = model(X)
    yPred = yPred.float()
    y = y.unsqueeze(1)

    loss = lossFn(yPred, y)
    loss.backward()
    optimizer.step()

    return loss.item()

def testStep(engine, batch):
    model.eval()
    with torch.no_grad():
        X, y = batch
        yPred = model(X)
    return yPred, y

df = pd.read_csv("../Data/Datasets/0.5.3-MRobust_logHenry.csv")
time = dt.datetime.now().strftime("%Y-%m-%d_%H.%M.%S") #For saving files 

X = np.float32(df.iloc[:, 5:].values)
y = np.float32(df["logHenry"].values)

xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2, random_state=42)

model = NeuralNetwork(xTrain.shape[1]).to(device)
print(model)

bat = 64

trainSet = training_set(xTrain, yTrain)
trainLoader = DataLoader(trainSet, batch_size = bat, shuffle=True)

testSet = training_set(xTest, yTest)
testLoader = DataLoader(testSet, batch_size = bat, shuffle=True)

lossFn = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.005)

trainer = Engine(trainStep)
evaluator = Engine(testStep)
MeanSquaredError().attach(evaluator, "MSE")

validateEvery = 5

@trainer.on(Events.EPOCH_COMPLETED(every=validateEvery)) # Occurs every nth epoch
def runValidation():
    evaluator.run(testLoader)
    print(f"Epoch: {trainer.state.epoch}, Validation MSE: {evaluator.state.metrics['MSE']}")

@trainer.on(Events.EPOCH_COMPLETED) #Occurs after every epoch
def logTrainingLoss(engine):
    print(f"Epoch: {engine.state.epoch}, Iter: [{engine.state.iteration}] Loss: {engine.state.output}")

def scoreFn(engine):
    valLoss = engine.state.metrics['MSE']
    return -valLoss

handler = EarlyStopping(patience=1, score_function=scoreFn, trainer=trainer)
evaluator.add_event_handler(Events.COMPLETED, handler)

tb_logger = TensorboardLogger(log_dir="tb-logger")
tb_logger.attach_output_handler(
    trainer,
    event_name=Events.EPOCH_COMPLETED,
    tag="training",
    output_transform=lambda x: x,
)

for tag, evaluator in [("training", trainer), ("validation", evaluator)]:
    tb_logger.attach_output_handler(
        evaluator,
        event_name=Events.EPOCH_COMPLETED,
        tag=tag,
        metric_names="all",
        global_step_transform=global_step_from_engine(trainer),
    )

trainer.run(trainLoader, max_epochs=30)
tb_logger.close()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=122, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)
Epoch: 1, Iter: [79] Loss: 476.8125
Epoch: 2, Iter: [158] Loss: 11.857799530029297
Epoch: 3, Iter: [237] Loss: 5.512316703796387
Epoch: 4, Iter: [316] Loss: 8.121548652648926
Epoch: 5, Validation MSE: 6.059586226276828
Epoch: 5, Iter: [395] Loss: 8.515018463134766
Epoch: 6, Iter: [474] Loss: 7.784921646118164
Epoch: 7, Iter: [553] Loss: 2.016859531402588
Epoch: 8, Iter: [632] Loss: 2.3240697383880615
Epoch: 9, Iter: [711] Loss: 2.6891112327575684


2024-08-02 14:22:05,478 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


Epoch: 10, Validation MSE: 6.27417884911566
Epoch: 10, Iter: [790] Loss: 7.62074613571167
